# VEST MI 2020 - US President and US Senate

In [36]:
import pandas as pd
import geopandas as gp
import numpy as np
import matplotlib.pyplot as plt

## Load VEST File

In [37]:
pwd

'/Users/peterhorton/Documents/RDH/pdv/pdv-mi/vest-mi-2020'

In [38]:
gdfv = gp.read_file('./raw_from_source/mi_2020/mi_2020.shp')
gdfv.head()

,PRECINCTID,COUNTYFIPS,cousubname,elexpre,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,G20PRENDEL,G20PRETBLA,G20USSRJAM,G20USSDPET,G20USSGSQU,G20USSNDER,G20USSTWIL,geometry
0,WP-001-01040-00001,001,Alcona township,001-ALCONA TOWNSHIP-0-0001,564,248,3,2,0,2,539,267,4,2,3,"POLYGON ((-83.29467 44.77346, -83.29577 44.773..."
1,WP-001-12460-00001,001,Caledonia township,001-CALEDONIA TOWNSHIP-0-0001,508,245,4,0,0,0,485,261,1,0,4,"POLYGON ((-83.64206 44.81382, -83.64578 44.813..."
2,WP-001-19320-00001,001,Curtis township,001-CURTIS TOWNSHIP-0-0001,486,238,2,1,0,1,456,240,5,4,10,"POLYGON ((-83.64530 44.51091, -83.64918 44.510..."
3,WP-001-34820-00001,001,Greenbush township,001-GREENBUSH TOWNSHIP-0-0001,560,302,9,1,0,1,531,322,4,5,6,"POLYGON ((-83.31858 44.51165, -83.32054 44.511..."
4,WP-001-35740-00001,001,Gustin township,001-GUSTIN TOWNSHIP-0-0001,317,112,9,0,0,0,306,122,1,0,6,"POLYGON ((-83.40227 44.59806, -83.41508 44.598..."


## Election Results

In [39]:
gdfv.columns

Index(['PRECINCTID', 'COUNTYFIPS', 'cousubname', 'elexpre', 'G20PRERTRU',
       'G20PREDBID', 'G20PRELJOR', 'G20PREGHAW', 'G20PRENDEL', 'G20PRETBLA',
       'G20USSRJAM', 'G20USSDPET', 'G20USSGSQU', 'G20USSNDER', 'G20USSTWIL',
       'geometry'],
      dtype='object')

### All candidates in election results file:

**2020:GEN:1:00000:0:-1417:WELLS:KASEY::NPA:
**2020:GEN:1:00000:0:-1416:HOEFLING:TOM::NPA:
**2020:GEN:1:00000:0:-1415:HUNTER:TARA RENEE::NPA:
**2020:GEN:1:00000:0:-1414:CARROLL:BRIAN:T.:NPA:
**2020:GEN:1:00000:0:-1413:SIMMONS:JADE::NPA:**
2020:GEN:1:00000:0:-1403:TRUMP:DONALD:J.:REP:
2020:GEN:1:00000:0:-1401:BIDEN:JOSEPH:R.:DEM:
2020:GEN:1:00000:0:-1398:DE LA FUENTE:ROCKY::NLP:
2020:GEN:1:00000:0:-1391:BLANKENSHIP:DON::UST:
2020:GEN:1:00000:0:-1385:JORGENSEN:JO::LIB:
2020:GEN:1:00000:0:-1373:HAWKINS:HOWIE::GRN:
**2020:GEN:5:00000:0:-1419:GADZINSKI:LEONARD:PAUL:NPA:
**2020:GEN:5:00000:0:-1400:CARR:ROBERT:WILLIAM:NPA:**
2020:GEN:5:00000:0:-1397:DERN:DOUG::NLP:
2020:GEN:5:00000:0:-1393:WILLIS:VALERIE:L.:UST:
2020:GEN:5:00000:0:-1365:SQUIER:MARCIA::GRN:
2020:GEN:5:00000:0:-1304:PETERS:GARY::DEM:
2020:GEN:5:00000:0:-1283:JAMES:JOHN::REP:

In [40]:
## Load + Process format
df = pd.read_csv('./raw_from_source/2020GEN/2020vote.txt', delimiter = '\t', header = None)

#columns defined in documentation
df.columns = ['election_year','election_type','office_code','district_code',
            'status_code','candidate_id','county_code','city_town_code','ward_number',
            'precinct_number','precinct_label','precinct_votes','na_col']

#Processing
#Filter to office code `1` for president
df = df[(df['office_code'] == 1)|(df['office_code'] == 5)]

#Have to replace NaNs in `precinct_label` with empty strings so that we can use it as an index in the pivot
df['precinct_label'] = df['precinct_label'].replace(np.nan, '', regex = True)

other_cols = [-1417, -1416, -1415, -1414, -1413, -1419, -1400]
df = df[df['candidate_id'].map(lambda x: x not in other_cols)]

## PIVOT RESULTS
df_pivot = df.pivot_table(index = ['county_code','city_town_code','ward_number','precinct_number','precinct_label'],
                         columns = ['candidate_id'],
                        values = ['precinct_votes'],
                         aggfunc = 'sum')
df_pivot.reset_index(inplace = True)

#Appears that VEST does not have a "other" column in this case...
#Using the file `2020name.txt` to figure out these candidate codes for candidates in election results not specified by vest: 
vote_cols_other = [( 'precinct_votes', -1417),
            ( 'precinct_votes', -1416),
            ( 'precinct_votes', -1415),
            ( 'precinct_votes', -1414),
            ( 'precinct_votes', -1413),
            ( 'precinct_votes', -1419),
            ( 'precinct_votes', -1400)]

df_pivot['G20PREOth'] = 0
for col in vote_cols_other:
    df_pivot['G20PREOth'] = df_pivot['G20PREOth'] + df_pivot[col]
    df_pivot.drop(col[1], axis = 1, level = 1, inplace = True)
    
df_pivot.reset_index(inplace = True)
df_pivot.columns = ['candidate_id','county_code','city_town_code','ward_number','precinct_number','precinct_label',
                    'G20PRERTRU','G20PREDBID', 'G20PRELJOR', 'G20PREGHAW', 'G20PRENDEL', 'G20PRETBLA','G20USSRJAM', 
                    'G20USSDPET', 'G20USSGSQU', 'G20USSNDER', 'G20USSTWIL', 'G20PREOth']

In [41]:
df_pivot

county_code city_town_code ward_number precinct_number  \
candidate_id                                                          
0                      1              2           0               1   
1                      1              4           0               1   
2                      1              6           0               1   
3                      1              8           0               1   
4                      1             10           0               1   
...                  ...            ...         ...             ...   
4918                  83             32           0               1   
4919                  83             52           0               1   
4920                  83             52           0               2   
4921                  83             52           0               3   
4922                  83             54           0               1   

             precinct_label precinct_votes                                 \
candidate_id                         -1403  -1401 -1398 -1397 -1393 -1391   
0                                    564.0  248.0   0.0   2.0   3.0   2.0   
1                                    508.0  245.0   0.0   0.0   4.0   0.0   
2                                    486.0  238.0   0.0   4.0  10.0   1.0   
3                                    560.0  302.0   0.0   5.0   6.0   1.0   
4                                    317.0  112.0   0.0   0.0   6.0   0.0   
...                     ...            ...    ...   ...   ...   ...   ...   
4918                                 490.0  166.0   0.0   5.0   3.0   2.0   
4919                                 973.0  600.0   2.0   4.0  19.0   1.0   
4920                                 881.0  652.0   0.0   6.0  17.0   4.0   
4921                                 958.0  659.0   0.0   1.0   8.0   3.0   
4922                                 392.0  162.0   0.0   1.0   7.0   0.0   

                                              
candidate_id -1385 -1373 -1365  -1304  -1283  
0              3.0   2.0   4.0  267.0  539.0  
1              4.0   0.0   1.0  261.0  485.0  
2              2.0   1.0   5.0  240.0  456.0  
3              9.0   1.0   4.0  322.0  531.0  
4              9.0   0.0   1.0  122.0  306.0  
...            ...   ...   ...    ...    ...  
4918           6.0   0.0   3.0  163.0  485.0  
4919          30.0   7.0  11.0  589.0  974.0  
4920          33.0   3.0  12.0  657.0  871.0  
4921          29.0   6.0  19.0  636.0  988.0  
4922          11.0   1.0   6.0  160.0  382.0  

[4923 rows x 16 columns]

In [42]:
df_pivot.columns = ['county_code','city_town_code','ward_number','precinct_number','precinct_label',
                    'G20PRERTRU','G20PREDBID', 'G20PRELJOR', 'G20PREGHAW', 'G20PRENDEL', 'G20PRETBLA','G20USSRJAM', 
                    'G20USSDPET', 'G20USSGSQU', 'G20USSNDER', 'G20USSTWIL']

In [43]:
df_pivot

,county_code,city_town_code,ward_number,precinct_number,precinct_label,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,G20PRENDEL,G20PRETBLA,G20USSRJAM,G20USSDPET,G20USSGSQU,G20USSNDER,G20USSTWIL
0,1,2,0,1,,564.0,248.0,0.0,2.0,3.0,2.0,3.0,2.0,4.0,267.0,539.0
1,1,4,0,1,,508.0,245.0,0.0,0.0,4.0,0.0,4.0,0.0,1.0,261.0,485.0
2,1,6,0,1,,486.0,238.0,0.0,4.0,10.0,1.0,2.0,1.0,5.0,240.0,456.0
3,1,8,0,1,,560.0,302.0,0.0,5.0,6.0,1.0,9.0,1.0,4.0,322.0,531.0
4,1,10,0,1,,317.0,112.0,0.0,0.0,6.0,0.0,9.0,0.0,1.0,122.0,306.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,83,32,0,1,,490.0,166.0,0.0,5.0,3.0,2.0,6.0,0.0,3.0,163.0,485.0
4919,83,52,0,1,,973.0,600.0,2.0,4.0,19.0,1.0,30.0,7.0,11.0,589.0,974.0
4920,83,52,0,2,,881.0,652.0,0.0,6.0,17.0,4.0,33.0,3.0,12.0,657.0,871.0
4921,83,52,0,3,,958.0,659.0,0.0,1.0,8.0,3.0,29.0,6.0,19.0,636.0,988.0


## Cleaning Election Results

In [44]:
pwd

'/Users/peterhorton/Documents/RDH/pdv/pdv-mi/vest-mi-2020'

In [45]:
#cty2016 = pd.read_csv('/Users/lilyfalk/sandbox/pdv-mi/vest-mi/vest_mi_2016_ar/raw-from-source/2016GEN/county.txt',delimiter = '\t', header = None)

In [46]:
cty2020 = pd.read_csv('raw_from_source/2020GEN/county.txt',delimiter = '\t', header = None)

In [47]:
#### Add County Name
#The SOS results have county codes that are not FIPS codes. Pulled the `county.txt` file to a Sheet and lined it up with county fips codes from Wikipedia. Loading in this file created in Sheets. 
county_names = pd.read_csv('./raw_from_source/county_codes.csv')

In [48]:
df_pivot = df_pivot.merge(county_names, on = 'county_code')

In [49]:
#### Add City/Township Name
#Loading in the `2016city.txt` file that has the corresponding names for the city codes in the election results file
city_codes = pd.read_csv('./raw_from_source/2020GEN/2020city.txt',delimiter='\t',header=None)
city_codes.columns = ['election_year','election_type','county_code','city_town_code','city_town_name','null']
city_codes.drop(['election_year','election_type','null'],axis=1,inplace=True)
df_pivot = df_pivot.merge(city_codes, on = ['county_code','city_town_code'])

In [50]:
df_pivot

,county_code,city_town_code,ward_number,precinct_number,precinct_label,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,G20PRENDEL,G20PRETBLA,G20USSRJAM,G20USSDPET,G20USSGSQU,G20USSNDER,G20USSTWIL,county_name,county_fips,city_town_name
0,1,2,0,1,,564.0,248.0,0.0,2.0,3.0,2.0,3.0,2.0,4.0,267.0,539.0,ALCONA,1,ALCONA TOWNSHIP
1,1,4,0,1,,508.0,245.0,0.0,0.0,4.0,0.0,4.0,0.0,1.0,261.0,485.0,ALCONA,1,CALEDONIA TOWNSHIP
2,1,6,0,1,,486.0,238.0,0.0,4.0,10.0,1.0,2.0,1.0,5.0,240.0,456.0,ALCONA,1,CURTIS TOWNSHIP
3,1,8,0,1,,560.0,302.0,0.0,5.0,6.0,1.0,9.0,1.0,4.0,322.0,531.0,ALCONA,1,GREENBUSH TOWNSHIP
4,1,10,0,1,,317.0,112.0,0.0,0.0,6.0,0.0,9.0,0.0,1.0,122.0,306.0,ALCONA,1,GUSTIN TOWNSHIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,83,32,0,1,,490.0,166.0,0.0,5.0,3.0,2.0,6.0,0.0,3.0,163.0,485.0,WEXFORD,165,WEXFORD TOWNSHIP
4919,83,52,0,1,,973.0,600.0,2.0,4.0,19.0,1.0,30.0,7.0,11.0,589.0,974.0,WEXFORD,165,CADILLAC CITY
4920,83,52,0,2,,881.0,652.0,0.0,6.0,17.0,4.0,33.0,3.0,12.0,657.0,871.0,WEXFORD,165,CADILLAC CITY
4921,83,52,0,3,,958.0,659.0,0.0,1.0,8.0,3.0,29.0,6.0,19.0,636.0,988.0,WEXFORD,165,CADILLAC CITY


## Validate Election Results at State and County Levels

In [51]:
#Function to check column/race totals
def colum_total_check(column_list, elections_df, vest_df):
    for val in column_list:
        vote_dif = elections_df[val].sum()-vest_df[val].sum()
        if (vote_dif == 0):
            print(val+": EQUAL", ' - total: ', str(elections_df[val].sum()))
        else:
            print(val+": DIFFERENCE OF " + str(vote_dif)+ " VOTES", '  rdh total: ', str(elections_df[val].sum()), ', vest total: ', str(vest_df[val].sum()))
            
#Function to check county totals
def county_total_check(column_list, elections_df, vest_df):
    print("Counties with differences printed below:")
    diff_counties=[]
    for i in column_list:
        diff = elections_df.groupby(["county_fips"]).sum()[i]-vest_df.groupby(["COUNTYFIPS"]).sum()[i]
        for val in diff[diff != 0].index.values.tolist():
            if val not in diff_counties:
                diff_counties.append(val)
        if len(diff[diff != 0]!=0):
            print(i)
            print(diff)
            #print(diff[diff != 0].to_string(header=False))
    print("")
    print("All other races in all counties are equal")

In [52]:
df_pivot['county_fips'] = df_pivot['county_fips'].map(lambda x: str(x).zfill(3))

In [53]:
df_pivot = df_pivot.reset_index()
df_pivot = df_pivot.fillna(0)

column_list = ['G20PRERTRU','G20PREDBID', 'G20PRELJOR', 'G20PREGHAW', 'G20PRENDEL', 'G20PRETBLA',
       'G20USSRJAM', 'G20USSDPET', 'G20USSGSQU', 'G20USSNDER', 'G20USSTWIL']

colum_total_check(column_list, df_pivot, gdfv)

G20PRERTRU: DIFFERENCE OF -625.0 VOTES   rdh total:  2649234.0 , vest total:  2649859
G20PREDBID: DIFFERENCE OF -2567.0 VOTES   rdh total:  2801469.0 , vest total:  2804036
G20PRELJOR: DIFFERENCE OF -57423.0 VOTES   rdh total:  2983.0 , vest total:  60406
G20PREGHAW: DIFFERENCE OF -631.0 VOTES   rdh total:  13087.0 , vest total:  13718
G20PRENDEL: DIFFERENCE OF 47581.0 VOTES   rdh total:  50566.0 , vest total:  2985
G20PRETBLA: DIFFERENCE OF 27.0 VOTES   rdh total:  7231.0 , vest total:  7204
G20USSRJAM: DIFFERENCE OF -2581865.0 VOTES   rdh total:  60356.0 , vest total:  2642221
G20USSDPET: DIFFERENCE OF -2720853.0 VOTES   rdh total:  13705.0 , vest total:  2734558
G20USSGSQU: DIFFERENCE OF -37.0 VOTES   rdh total:  39180.0 , vest total:  39217
G20USSNDER: DIFFERENCE OF 2718948.0 VOTES   rdh total:  2732041.0 , vest total:  13093
G20USSTWIL: DIFFERENCE OF 2591016.0 VOTES   rdh total:  2641612.0 , vest total:  50596


In [54]:
df_pivot.columns

Index(['index', 'county_code', 'city_town_code', 'ward_number',
       'precinct_number', 'precinct_label', 'G20PRERTRU', 'G20PREDBID',
       'G20PRELJOR', 'G20PREGHAW', 'G20PRENDEL', 'G20PRETBLA', 'G20USSRJAM',
       'G20USSDPET', 'G20USSGSQU', 'G20USSNDER', 'G20USSTWIL', 'county_name',
       'county_fips', 'city_town_name'],
      dtype='object')

In [55]:
set(gdfv['COUNTYFIPS']) - set(df_pivot['county_fips'])

set()

### NEW ADDITION: Changing the naming of the columns to better match what VEST has

From the above, it seemed like some of the column names had been swapped

In [56]:
df_pivot.columns = ['index', 'county_code', 'city_town_code', 'ward_number',
       'precinct_number', 'precinct_label', 'G20PRERTRU', 'G20PREDBID',
       'G20PRENDEL', 'G20PREGHAW', 'G20PRELJOR', 'G20PRETBLA', 'G20USSTWIL',
       'G20USSNDER', 'G20USSGSQU', 'G20USSDPET', 'G20USSRJAM', 'county_name',
       'county_fips', 'city_town_name']

colum_total_check(column_list, df_pivot, gdfv)

G20PRERTRU: DIFFERENCE OF -625.0 VOTES   rdh total:  2649234.0 , vest total:  2649859
G20PREDBID: DIFFERENCE OF -2567.0 VOTES   rdh total:  2801469.0 , vest total:  2804036
G20PRELJOR: DIFFERENCE OF -9840.0 VOTES   rdh total:  50566.0 , vest total:  60406
G20PREGHAW: DIFFERENCE OF -631.0 VOTES   rdh total:  13087.0 , vest total:  13718
G20PRENDEL: DIFFERENCE OF -2.0 VOTES   rdh total:  2983.0 , vest total:  2985
G20PRETBLA: DIFFERENCE OF 27.0 VOTES   rdh total:  7231.0 , vest total:  7204
G20USSRJAM: DIFFERENCE OF -609.0 VOTES   rdh total:  2641612.0 , vest total:  2642221
G20USSDPET: DIFFERENCE OF -2517.0 VOTES   rdh total:  2732041.0 , vest total:  2734558
G20USSGSQU: DIFFERENCE OF -37.0 VOTES   rdh total:  39180.0 , vest total:  39217
G20USSNDER: DIFFERENCE OF 612.0 VOTES   rdh total:  13705.0 , vest total:  13093
G20USSTWIL: DIFFERENCE OF 9760.0 VOTES   rdh total:  60356.0 , vest total:  50596


Looking into the total number of votes

In [59]:
tot_votes_1 = 0
tot_votes_2 = 0
for race in column_list:
    tot_votes_1+=sum(df_pivot[race])
    tot_votes_2+=sum(gdfv[race])
    
print(tot_votes_1-tot_votes_2)

-6429.0



## Looking for unique ids

In [20]:
(gdfv['COUNTYFIPS']+gdfv['cousubname'].str.lower()+gdfv['PRECINCTID'].str.slice(start = 13)).nunique()

4756

In [21]:
gdfv.shape

(4756, 16)

In [23]:
(df_pivot['county_fips'].astype(str) + df_pivot['city_town_name'] + df_pivot['ward_number'].astype(str) + df_pivot['precinct_number'].astype(str)).nunique()

4907

In [24]:
df_pivot.shape

(4923, 20)

In [25]:
df_pivot

,index,county_code,city_town_code,ward_number,precinct_number,precinct_label,G20PRERTRU,G20PREDBID,G20PRELJOR,G20PREGHAW,G20PRENDEL,G20PRETBLA,G20USSRJAM,G20USSDPET,G20USSGSQU,G20USSNDER,G20USSTWIL,county_name,county_fips,city_town_name
0,0,1,2,0,1,,564.0,248.0,0.0,2.0,3.0,2.0,3.0,2.0,4.0,267.0,539.0,ALCONA,001,ALCONA TOWNSHIP
1,1,1,4,0,1,,508.0,245.0,0.0,0.0,4.0,0.0,4.0,0.0,1.0,261.0,485.0,ALCONA,001,CALEDONIA TOWNSHIP
2,2,1,6,0,1,,486.0,238.0,0.0,4.0,10.0,1.0,2.0,1.0,5.0,240.0,456.0,ALCONA,001,CURTIS TOWNSHIP
3,3,1,8,0,1,,560.0,302.0,0.0,5.0,6.0,1.0,9.0,1.0,4.0,322.0,531.0,ALCONA,001,GREENBUSH TOWNSHIP
4,4,1,10,0,1,,317.0,112.0,0.0,0.0,6.0,0.0,9.0,0.0,1.0,122.0,306.0,ALCONA,001,GUSTIN TOWNSHIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4918,4918,83,32,0,1,,490.0,166.0,0.0,5.0,3.0,2.0,6.0,0.0,3.0,163.0,485.0,WEXFORD,165,WEXFORD TOWNSHIP
4919,4919,83,52,0,1,,973.0,600.0,2.0,4.0,19.0,1.0,30.0,7.0,11.0,589.0,974.0,WEXFORD,165,CADILLAC CITY
4920,4920,83,52,0,2,,881.0,652.0,0.0,6.0,17.0,4.0,33.0,3.0,12.0,657.0,871.0,WEXFORD,165,CADILLAC CITY
4921,4921,83,52,0,3,,958.0,659.0,0.0,1.0,8.0,3.0,29.0,6.0,19.0,636.0,988.0,WEXFORD,165,CADILLAC CITY


In [26]:
len(set(df_pivot.city_town_name.str.lower()) - set(gdfv.cousubname.str.lower()))

110

In [27]:
len(set(gdfv.cousubname.str.lower()) - set(df_pivot.city_town_name.str.lower()))

140

In [28]:
gdfv['idtest1'] = (gdfv['COUNTYFIPS']+gdfv['cousubname'].str.lower()+gdfv['PRECINCTID'].str.slice(start = 13))

In [29]:
df_pivot['idtest1'] = (df_pivot['county_fips'].astype(str).str.zfill(3) + df_pivot['city_town_name'].str.lower() + df_pivot['ward_number'].astype(str).str.zfill(2) + df_pivot['precinct_number'].astype(str).str.zfill(3))

In [30]:
len(set(gdfv['idtest1']) - set(df_pivot['idtest1']))

966

In [31]:
len(set(df_pivot['idtest1']) - set(gdfv['idtest1']))

1117

Using the above as a preliminary precinct id, can export and hand match in Excel.

Still have the issue of vote totals not matching by column or county. How to account for??

## Shapefile

In [32]:
shp = gp.read_file('./raw_from_source/2020_Voting_Precincts/2020_Voting_Precincts.shp')
crs = gdfv.crs
shp = shp.to_crs(crs)
shp.head()

,PRECINCTID,ELECTIONYE,COUNTYFIPS,MCDFIPS,WARD,PRECINCT,OBJECTID_1,OBJECTID,NAME,STATEFP,VTDST,FUNCSTAT,VTDI,LSAD,ShapeSTAre,ShapeSTLen,geometry
0,WP-001-01040-00001,2020,001,01040,00,001,1,1,0010104000001,26,001001,N,A,V1,1.701076e+08,73869.558749,"POLYGON ((-83.29466 44.77345, -83.29577 44.773..."
1,WP-001-12460-00001,2020,001,12460,00,001,2,2,0011246000001,26,001002,N,A,V1,1.880860e+08,76737.144518,"POLYGON ((-83.64206 44.81382, -83.64578 44.813..."
2,WP-001-19320-00001,2020,001,19320,00,001,3,3,0011932000001,26,001003,N,A,V1,1.833231e+08,57470.528359,"POLYGON ((-83.64529 44.51090, -83.64917 44.510..."
3,WP-001-34820-00001,2020,001,34820,00,001,4,4,0013482000001,26,001004,N,A,V1,6.767958e+07,32987.191075,"POLYGON ((-83.31858 44.51164, -83.32053 44.511..."
4,WP-001-35740-00001,2020,001,35740,00,001,5,5,0013574000001,26,001005,N,A,V1,9.281010e+07,38642.299469,"POLYGON ((-83.40226 44.59806, -83.41507 44.598..."
